In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [3]:
df = pd.read_excel('Opta.xls')

In [5]:
rows_missing = df[df['Player Forename'].isnull()]
selected_clumns = ['Player ID', 'Player Surname', 'Player Forename', 'Team']
rows_missing = rows_missing[selected_clumns]
unique_missing = rows_missing['Player Surname'].unique().tolist()

In [6]:

# Filter rows with missing values in the 'Player Forename' column and unique Player Surnames
rows_to_transform = df[df['Player Forename'].isnull() & df['Player Surname'].isin(unique_missing)]

# Define a function to split names based on the cases you mentioned
def split_name(full_name):
    if ' ' not in full_name:  # Single name
        return ('', full_name)
    else:
        parts = full_name.split(' ')
        if len(parts) == 2:  # Western name
            return (parts[0], parts[1])
        else:  # Asian name
            return (parts[1], parts[0])  # Swap the positions for Asian names

# Apply the split_name function and create two new columns
split_names = rows_to_transform['Player Surname'].apply(split_name).apply(pd.Series)
split_names.columns = ['Player Forename', 'Player Surname']

# Update df with the modified rows
df.update(split_names)

In [7]:
# List of the Asian names that were swapped
specific_names = ['Ji Dong-Won', 'Lee Chung-Yong', 'Park Chu-Young', 'Park Ji-Sung']

# Iterate through the DataFrame and swap values for the specific names
for name in specific_names:
    surname, forename = name.split()
    df.loc[df['Player Surname'] == forename, 'Player Surname'], df.loc[df['Player Forename'] == surname, 'Player Forename'] = surname, forename

In [8]:
# List of columns to exclude from summation
exclude_columns = [
    'Index', 'Date', 'Player ID', 'Player Surname', 'Player Forename', 'Team',
    'Team Id', 'Opposition', 'Opposition id', 'Venue', 'Position Id',
    'Team Formation', 'Position in Formation', 'CompId', 'SeasId'
]

# Identify columns to sum (exclude those in the exclude_columns list)
sum_columns = [col for col in df.columns if col not in exclude_columns]

# Group by Player ID and sum the data to get stats per player for the whole season
summarized_df = df.groupby(['Player ID', 'Player Forename', 'Player Surname'])[sum_columns].sum().reset_index()

In [9]:
ylabel = pd.read_excel('Transfermarkt.xlsx')

In [10]:
# Merge names in order to merge both datasets on the full player name
summarized_df['player_name'] = summarized_df['Player Forename'] + ' ' + summarized_df['Player Surname']

In [11]:
# Some names differ, this variable contains the players where the name differs between the 2 datasets
missing_names = pd.read_excel('Missing Players.xlsx')

In [12]:
# Merge the dataframes based on Player Forename and Player Surname
temp_df = pd.merge(summarized_df, missing_names, on=['Player Forename', 'Player Surname'], how='left', suffixes=('', '_missing'))

# Update player_name in summarized_df with values from 'ylabel name' in missing_names
summarized_df['player_name'] = temp_df['ylabel name'].combine_first(temp_df['player_name'])

In [13]:
# Merge summarized_df with ylabel

# Filter ylabel based on the condition season_start_year == 2011
filtered_ylabel = ylabel[ylabel['season_start_year'] == 2011]


# Merge the dataframes based on player_name and the filtered ylabel
merged_df = pd.merge(summarized_df, filtered_ylabel[['player_name', 'player_position', 'player_dob', 'player_nationality',
                                                     'player_height_mtrs', 'player_foot', 'player_market_value_euro']],
                     on='player_name', how='left')





In [14]:
## Adjust player position
merged_df.loc[merged_df['player_name'] == 'David Jones', 'player_position'] = 'Central Midfield'

In [15]:
## Transform Date of Birth to Age on the end of the 2011/2012 season to use it for plots
from datetime import datetime
merged_df['player_dob'] = pd.to_datetime(merged_df['player_dob'])

## end of season date 
end_date = datetime(2012, 5, 13)

# Age on end_date
merged_df['age'] = merged_df['player_dob'].apply(lambda dob: (end_date - dob).days // 365)

In [16]:
# Create shots on target excl goals
merged_df['Shots on target excl goals'] = merged_df['Shots On Target inc goals'] - merged_df[ 'Goals']

In [19]:
merged_df.to_csv(r"C:\Users\damon\OneDrive\Documenten\Tilburg University\Master DSS\Thesis\merged_data.csv", index=False)